In [ ]:
!pip install -q scikit-multiflow

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skmultiflow.drift_detection import ADWIN, DDM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import warnings
import scipy.io
import pickle
import sys
import gc
import os
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
results_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/Results'
feature_selection_results = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results'
feature_selection_issam = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_issam'
sys.path.insert(0,code_path)
from oselm import set_use_know
from SharedFunctions import main,generate_oselm_models

In [ ]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','Recurrent_Drift.csv', 'Blip_Drift.csv', 'Incremental_Drift.csv',
             'Gradual_Drift.csv', 'Sudden_Drift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))

In [ ]:
data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','Recurrent_Drift', 'Blip_Drift', 'Incremental_Drift','Gradual_Drift', 'Sudden_Drift']

In [ ]:
# # use_know = True
# # set_use_know(use_know)
# # path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')  # IssamDFS
# # os.makedirs(path, exist_ok=True)
# # for f_name,d_name in zip(filenames,data_name):
# #   DFS_results_path = os.path.join(feature_selection_results,d_name)
# #   main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
# #       feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,DFS_results_path=DFS_results_path)

In [ ]:
DATA_NUMBER=0
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,is_synthetic=False)

In [ ]:
DATA_NUMBER=1
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,is_synthetic=False,chunk_size = 1000)

In [ ]:
DATA_NUMBER=2
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,is_synthetic=False,chunk_size = 3500)

In [ ]:
DATA_NUMBER=3
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,is_synthetic=False,chunk_size = 1000)

In [ ]:
DATA_NUMBER=4
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=5
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=6
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=7
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=8
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS') # IssamDFS
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results, d_name) # feature_selection_issam
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)